# Naive Bayes Decision

In [1]:
import pandas as pd
import numpy as np

In [2]:
values: dict[int, (np.array, np.array)] = dict()

In [3]:
m = np.array([1, 2, 1, 2, 1])
s = np.array([0.1, 0.1, 0.1, 0.1, 0.1])

In [4]:
values[0] = [m, s, 4]
values[1] = [m + 1, s, 2]

In [5]:
values

{0: [array([1, 2, 1, 2, 1]), array([0.1, 0.1, 0.1, 0.1, 0.1]), 4],
 1: [array([2, 3, 2, 3, 2]), array([0.1, 0.1, 0.1, 0.1, 0.1]), 2]}

In [6]:
X = pd.DataFrame(np.array([[1, 1, 1, 1, 1], [2, 3, 3, 3, 3], [0, 0, 0, 0, 0], [3, 4, 3, 3, 3]]))
Y = [0, 1, 0, 1]

In [7]:
kp = {k: np.sum(((X - values[k][0]) / values[k][1]) ** 2, axis=1) for k in values}

In [8]:
kp

{0: 0     200.0
 1    1100.0
 2    1100.0
 3    1700.0
 dtype: float64,
 1: 0    1100.0
 1     200.0
 2    3000.0
 3     400.0
 dtype: float64}

In [9]:
yp = []
for i in range(len(X)):
    hk = 0
    hv = float('inf')
    for k in kp:
        if kp[k][i] < hv:
            hk = k
            hv = kp[k][i]
    yp.append(hk)

In [10]:
yp

[0, 1, 0, 1]

In [11]:
df = X.copy()
df['Y'] = Y

In [12]:
np.unique(Y)

array([0, 1])

In [13]:
evalvalues = {}
yarr = np.array(Y)
ks = np.unique(yarr)
df = X.copy()
for k in ks:
    evalvalues[k] = [np.array(np.mean(df[yarr == k], axis=0)), np.array(np.std(df[yarr == k], axis=0)), sum(yarr == k)]
evalvalues

{0: [array([0.5, 0.5, 0.5, 0.5, 0.5]), array([0.5, 0.5, 0.5, 0.5, 0.5]), 2],
 1: [array([2.5, 3.5, 3. , 3. , 3. ]), array([0.5, 0.5, 0. , 0. , 0. ]), 2]}

# Test Federated Bayes Decision

In [14]:
from fbd import FBD

### Initiation with Values

In [15]:
fbd = FBD(values)

In [16]:
fbd.values

{0: [array([1, 2, 1, 2, 1]), array([0.1, 0.1, 0.1, 0.1, 0.1]), 4],
 1: [array([2, 3, 2, 3, 2]), array([0.1, 0.1, 0.1, 0.1, 0.1]), 2]}

In [17]:
fbd.predict(X)

[0, 1, 0, 1]

### Initiation with Training

In [18]:
nfbd = fbd.copy()

In [19]:
nfbd.fit(X, Y)

In [20]:
nfbd.values

{0: [array([0.5, 0.5, 0.5, 0.5, 0.5]), array([0.5, 0.5, 0.5, 0.5, 0.5]), 2],
 1: [array([2.5, 3.5, 3. , 3. , 3. ]), array([0.5, 0.5, 0. , 0. , 0. ]), 2]}

In [21]:
nfbd.predict(X)

[0, 1, 0, 1]

## Merge Bayes Decision

In [22]:
fbd.merge(nfbd)

In [23]:
fbd.values

{0: (array([0.83333333, 1.5       , 0.83333333, 1.5       , 0.83333333]),
  array([0.32988511, 0.71261263, 0.32988511, 0.71261263, 0.32988511]),
  6),
 1: (array([2.25, 3.25, 2.5 , 3.  , 2.5 ]),
  array([0.41413762, 0.41413762, 0.50495098, 0.05      , 0.50495098]),
  4)}

In [24]:
fbd.predict(X)

[0, 1, 0, 1]

## Extra Prediction 1 Function

In [25]:
X1 = X.iloc[0]

In [26]:
def probability_one(X1, vmean, vstd):
    return np.exp(-np.sum(((X1 - vmean) / vstd) ** 2)) / np.sqrt(sum(vstd ** 2) * 2 * np.pi)

In [27]:
[probability_one(X1, fbd.values[k][0], fbd.values[k][1]) for k in fbd.values]

[0.059819305727353674, 0.0]

In [28]:
def predict_one(X1):
    return {
        k: fbd.values[k][2] * probability_one(X1, fbd.values[k][0], fbd.values[k][1]) for k in fbd.values
    }

In [29]:
for _, X1 in X.iterrows():
    print(predict_one(X1))

{0: 0.35891583436412205, 1: 0.0}
{0: 3.6786652446937237e-47, 1: 0.11713993756086295}
{0: 1.4208134733865976e-12, 1: 0.0}
{0: 6.953146244475548e-64, 1: 0.00034397176628050947}


In [30]:
for _, X1 in X.iterrows():
    print(fbd.predict_one(X1))

{0: 0.35891583436412205, 1: 0.0}
{0: 3.6786652446937237e-47, 1: 0.11713993756086295}
{0: 1.4208134733865976e-12, 1: 0.0}
{0: 6.953146244475548e-64, 1: 0.00034397176628050947}
